# Blending ensembling
 - [Predictores base](#Predictores-base)
 - [Separación del 10%](#Separación-del-10%)
 - [Predicción para el Blending](#Predicción-para-el-Blending)
 - [Modelo Blender](#Modelo-Blender)
 - [Predicción del set testing con los modelos base](#Predicción-del-set-testing-con-los-modelos-base)
 - [Predicción del set testing con el blender](#Predicción-del-set-testing-con-el-blender)

Repito con un super set de entrenamiento:

 - [Super set de entrenamiento](#Super-set-de-entrenamiento)
 - [Separación del 10% del super-set](#Separación-del-10%-del-super-set)
 - [Predicción para el super Blending](#Predicción-para-el-super-Blending)
 - [Modelo Blender con el super-set](#Modelo-Blender-con-el-super-set)
 - [Predicción del set de testing con los modelos base entrenados con el super-set de entrenamiento](#Predicción-del-set-de-testing-con-los-modelos-base-entrenados-con-el-super-set-de-entrenamiento)

In [1]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == get_clean_df()['person'].unique().shape[0])

Cargo el set de entrenamiento.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
train = labels_with_features.drop('label', axis=1)
train_target = labels_with_features['label']

## Predictores base

En esta sección vamos a preparar los predictores base a utilizar. Estos son los mismos que se encuentran en la carpeta *Algoritmos de ML*.

In [6]:
base_predictors = []

### AdaBoost


In [7]:
param = {
    'n_estimators': 128,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

base_predictors.append(AdaBoostRegressor(**param))

### Random Forest

In [8]:
param = {
    'bootstrap': True,
    'max_depth': 10,
    'max_features': 81,
    'min_samples_leaf': 49,
    'min_samples_split': 8,
    'n_estimators': 56
}

base_predictors.append(RandomForestRegressor(**param))

### DecisionTree

In [9]:
base_predictors.append(DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))

### Xgboost

In [10]:
param = {
    'objective': 'reg:logistic',
    'colsample_bylevel': 0.6605668347627213,
    'colsample_bytree': 0.5279014819087092,
    'min_child_weight': 4.302125582335056,
    'learning_rate': 0.15803667962605694,
    'max_delta_step': 7.592652591386328,
    'n_estimators': 65,
    'reg_lambda': 1.1181195507921775,
    'max_depth': 9,
    'silent': True,
    'subsample': 0.43744176565530823,
    'reg_alpha': 3.845311207046479,
    'gamma': 6.219264874528072
}

base_predictors.append(XGBRegressor(**param))

## Separación del 10%

Separo un 10% del set de entrenamiento para luego entrenar el blending.

In [11]:
train_90, train_10, train_90_target, train_10_target = train_test_split(train, train_target, test_size=0.1, stratify=train_target)
assert(train_90.shape[0] == train_90_target.shape[0])
assert(train_10.shape[0] == train_10_target.shape[0])

## Predicción para el Blending

Ahora vamos a realizar una predicción por cada predictor base del 10% separado, utilizando el 90% del set de entrenamiento (separando este 90% en sets de entrenamiento y validacion para optimizar cada modelo). Estas predicciones las vamos a agregar a una copia del set de entrenamiento (*train_blending_predictions*).

**Nota**: falta hacer el cross validation con el 90% para encontrar los parámetros óptimos. Actualmento los parámetros están optimizados para el 100%. Esto implica una filtración.

In [12]:
train_10_with_base_predictions = train_10.copy()
for predictor in base_predictors:
    train_10_with_base_predictions[predictor.__class__.__name__] = np.nan

In [13]:
%%time
for predictor in base_predictors:
    predictor.fit(train_90, train_90_target) # train
    train_10_with_base_predictions[predictor.__class__.__name__] = predictor.predict(train_10) # predict

CPU times: user 44.2 s, sys: 52 ms, total: 44.3 s
Wall time: 1min 59s


In [14]:
train_10_with_base_predictions.head(3)

screen_resolution_height mean  screen_resolution_width mean  \
person                                                                  
fcb5fb33                          480.0                    320.000000   
51fdbb43                          900.0                   1600.000000   
177bc00b                          768.0                   1109.333333   

          screen_resolution_height std  screen_resolution_width std  \
person                                                                
fcb5fb33                           0.0                     0.000000   
51fdbb43                           0.0                     0.000000   
177bc00b                           0.0                   147.801669   

          ad campaign hit  brand listing  checkout  conversion  \
person                                                           
fcb5fb33              0.0            0.0       1.0         0.0   
51fdbb43              3.0            0.0       1.0         0.0   
177bc00b              8.0           15.0       3.0         0.0   

          generic listing  lead  search engine hit  searched products  \
person                                                                  
fcb5fb33              5.0   0.0                0.0                2.0   
51fdbb43              0.0   0.0                0.0                0.0   
177bc00b              1.0   0.0                2.0                0.0   

              ...       Sunday  Thursday  Tuesday  Wednesday  tarde  \
person        ...                                                     
fcb5fb33      ...          0.0       0.0      0.0        0.0      0   
51fdbb43      ...          0.0       0.0      0.0        0.0      0   
177bc00b      ...          0.0       0.0      0.0        0.0      0   

          madrugada  maniana  noche  AdaBoostRegressor  RandomForestRegressor  \
person                                                                          
fcb5fb33          0        0      0           0.058259               0.006396   
51fdbb43          0        0      0           0.038176               0.001690   
177bc00b          0        0      0           0.447653               0.324053   

          DecisionTreeRegressor  XGBRegressor  
person                                         
fcb5fb33               0.008419      0.011864  
51fdbb43               0.008419      0.006298  
177bc00b               0.212080      0.259470  

[3 rows x 132 columns]

## Modelo Blender

Entrenamos un modelo blender que use las predicciones de los modelos base (el 10% que separamos) para realizar las predicciones del set de tesing (el que está en kaggle).

**Nota**: en este paso podríamos agregar algunos de los features originales.

In [15]:
blend_train = train_10_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
blend_train.head(3)

,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor,XGBRegressor
person,,,,
fcb5fb33,0.058259,0.006396,0.008419,0.011864
51fdbb43,0.038176,0.001690,0.008419,0.006298
177bc00b,0.447653,0.324053,0.212080,0.259470


In [16]:
param = {
    'solver': 'lbfgs'
}
blender = LogisticRegression(**param)

In [17]:
%%time
scores = cross_val_score(blender, blend_train, train_10_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.858502 (+/- 0.123263)
CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 360 ms


In [17]:
blender.fit(blend_train, train_10_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

## Predicción del set testing con los modelos base

In [18]:
testing = labels_to_predict.merge(df_features, how='inner', on='person')
assert(testing.shape[0] == labels_to_predict.shape[0])

In [19]:
testing_with_base_predictions = testing.copy()
for predictor in base_predictors:
    testing_with_base_predictions[predictor.__class__.__name__] = np.nan

In [20]:
%%time
for predictor in base_predictors:
    # como warm_start=False cada vez que llamo fit, el modelo se reinicia
    predictor.fit(train, train_target) # train
    testing_with_base_predictions[predictor.__class__.__name__] = predictor.predict(testing) # predict

CPU times: user 53.3 s, sys: 133 ms, total: 53.5 s
Wall time: 53.5 s


## Predicción del set testing con el blender

In [21]:
testing_with_base_predictions_for_blender = testing_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
testing_with_base_predictions_for_blender.head(3)

,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor
person,,,
4886f805,0.038229,0.002735,0.007434
0297fc1e,0.143778,0.064331,0.039040
2d681dd8,0.038229,0.004466,0.007434


Predecimos probabilidades.

In [22]:
# esto es muy específico de logistic reggresion
testing_target = pd.DataFrame(data=blender.predict_proba(testing_with_base_predictions_for_blender))[[1]]
testing_target.index = testing_with_base_predictions_for_blender.index
testing_target.index.name = 'person'
testing_target.columns = ['label']

Vamos a probar prediciendo labels.

In [23]:
testing_target = pd.DataFrame(data=blender.predict(testing_with_base_predictions_for_blender))
testing_target.index = testing_with_base_predictions_for_blender.index
testing_target.index.name = 'person'
testing_target.columns = ['label']

In [24]:
# si queremos aplicar blending normal:
# testing_target.to_csv(predicciones_csv)

## Super-set de entrenamiento

Juntamos el set de testing y el de entrenamiento en un nuevo super-set.

In [25]:
super_train = train.append(testing)
super_train_target = train_target.append(testing_target['label'])
assert(super_train.shape[0] == super_train_target.shape[0])

## Separación del 10% del super-set

In [26]:
# ya no puedo usar stratify porque en super_train_target tengo 0, 1 y probabilidades
super_train_90, super_train_10, super_train_90_target, super_train_10_target = train_test_split(super_train, super_train_target, test_size=0.1)
assert(super_train_90.shape[0] == super_train_90_target.shape[0])
assert(super_train_10.shape[0] == super_train_10_target.shape[0])

## Predicción para el super Blending

Ahora vamos a realizar una predicción por cada predictor base del 10% separado, utilizando el 90% del super-set de entrenamiento (separando este 90% en sets de entrenamiento y validacion para optimizar cada modelo). Estas predicciones las vamos a agregar a una copia del set de entrenamiento (*super_train_blending_predictions*).

**Nota**: falta hacer el cross validation con el 90% para encontrar los parámetros óptimos. Actualmento los parámetros están optimizados para el 100% del set de entrenamiento original. Esto implica una filtración.

In [27]:
super_train_10_with_base_predictions = super_train_10.copy()
for predictor in base_predictors:
    super_train_10_with_base_predictions[predictor.__class__.__name__] = np.nan

In [28]:
%%time
for predictor in base_predictors:
    predictor.fit(super_train_90, super_train_90_target) # train
    super_train_10_with_base_predictions[predictor.__class__.__name__] = predictor.predict(super_train_10) # predict

CPU times: user 1min 56s, sys: 280 ms, total: 1min 56s
Wall time: 1min 56s


## Modelo Blender con el super-set

Necesitamos un modelo en el cual los targets puedan ser floats, por eso no usamos LogisticRegression. En este caso decidimos utilizar xgboost.

In [29]:
super_blend_train = super_train_10_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
super_blend_train.head(3)

,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor
person,,,
a8d906b0,0.409028,0.167797,0.102187
42316be2,0.029734,0.001610,0.004667
37135eae,0.063227,0.006322,0.004667


In [30]:
param = {
#     'n_estimators': 'lbfgs'
    'solver': 'lbfgs'
}
super_blender = LogisticRegression(**param)

In [31]:
%%time
scores = cross_val_score(super_blender, super_blend_train, super_train_10_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.872336 (+/- 0.096028)
CPU times: user 370 ms, sys: 50 ms, total: 420 ms
Wall time: 112 ms


In [32]:
super_blender.fit(super_blend_train, super_train_10_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

## Predicción del set de testing con los modelos base entrenados con el super-set de entrenamiento

In [33]:
super_testing_with_base_predictions = testing.copy()
for predictor in base_predictors:
    super_testing_with_base_predictions[predictor.__class__.__name__] = np.nan

In [34]:
%%time
for predictor in base_predictors:
    # como warm_start=False cada vez que llamo fit, el modelo se reinicia
    predictor.fit(super_train, super_train_target) # train
    super_testing_with_base_predictions[predictor.__class__.__name__] = predictor.predict(testing) # predict

CPU times: user 2min 7s, sys: 365 ms, total: 2min 7s
Wall time: 2min 7s


## Predicción del set testing con el super blender

In [35]:
testing_with_base_predictions_for_super_blender = super_testing_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
testing_with_base_predictions_for_super_blender.head(3)

,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor
person,,,
4886f805,0.031315,0.001652,0.003917
0297fc1e,0.140220,0.028422,0.023855
2d681dd8,0.031315,0.002166,0.003917


In [36]:
predictions = super_blender.predict(testing_with_base_predictions_for_super_blender)
predictions = pd.DataFrame(data=predictions, index=testing_with_base_predictions_for_super_blender.index)
predictions.columns = ['label']
predictions.head(3)

,label
person,
4886f805,0
0297fc1e,0
2d681dd8,0


In [37]:
# predictions.to_csv(predicciones_csv)